<a href="https://colab.research.google.com/github/Sohitga014/econometrics-data-/blob/main/hp_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
hp=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Python docs/hp_ML.csv')
hp.sample()

,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country
684,14-05-2014 00:00,267800.0,2,1.0,700,6000,1.0,0,0,4,700,0,1949,1985,16824 11th Pl NE,Shoreline,WA 98155,USA


In [ ]:
hp.shape

(4600, 18)

In [ ]:
hp.isnull().sum()

date             0
price            0
bedrooms         0
bathrooms        0
sqft_living      0
sqft_lot         0
floors           0
waterfront       0
view             0
condition        0
sqft_above       0
sqft_basement    0
yr_built         0
yr_renovated     0
street           0
city             0
statezip         0
country          0
dtype: int64

In [ ]:
to_drop=['date','street','statezip','country']
hp_d=hp.drop(to_drop,axis=1)

In [ ]:
hp_d.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,city
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,1955,2005,Shoreline
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,1921,0,Seattle
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,1966,0,Kent
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,1963,0,Bellevue
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,1976,1992,Redmond


In [ ]:
hp_d['house_age']=[2022-yr_built for yr_built in hp_d['yr_built']]

In [ ]:
hp_d['was_renovated'] = [1 if yr_renovated != 0 else 0 for yr_renovated in hp_d['yr_renovated']]
hp_d['was_renovated_10_yrs'] = [1 if (2022 - yr_renovated) <= 10 else 0 for yr_renovated in hp_d['yr_renovated']]
hp_d['was_renovated_30_yrs'] = [1 if 10 < (2022 - yr_renovated) <= 30 else 0 for yr_renovated in hp_d['yr_renovated']]

In [ ]:
hp_2d=hp_d.drop(['yr_built','yr_renovated'],axis=1)

In [ ]:
hp_2d.head()

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,city,house_age,was_renovated,was_renovated_10_yrs,was_renovated_30_yrs
0,313000.0,3,1.50,1340,7912,1.5,0,0,3,1340,0,Shoreline,67,1,0,1
1,2384000.0,5,2.50,3650,9050,2.0,0,4,5,3370,280,Seattle,101,0,0,0
2,342000.0,3,2.00,1930,11947,1.0,0,0,4,1930,0,Kent,56,0,0,0
3,420000.0,3,2.25,2000,8030,1.0,0,0,4,1000,1000,Bellevue,59,0,0,0
4,550000.0,4,2.50,1940,10500,1.0,0,0,4,1140,800,Redmond,46,1,0,1


In [ ]:
hp_2d['city'].value_counts()

Seattle                1573
Renton                  293
Bellevue                286
Redmond                 235
Issaquah                187
Kirkland                187
Kent                    185
Auburn                  176
Sammamish               175
Federal Way             148
Shoreline               123
Woodinville             115
Maple Valley             96
Mercer Island            86
Burien                   74
Snoqualmie               71
Kenmore                  66
Des Moines               58
North Bend               50
Covington                43
Duvall                   42
Lake Forest Park         36
Bothell                  33
Newcastle                33
SeaTac                   29
Tukwila                  29
Vashon                   29
Enumclaw                 28
Carnation                22
Normandy Park            18
Clyde Hill               11
Medina                   11
Fall City                11
Black Diamond             9
Ravensdale                7
Pacific             

In [ ]:
def remap_location(data: pd.DataFrame, location: str, threshold: int = 50) -> str:
    if len(hp_2d[hp_2d['city'] == location]) < threshold:
        return 'Rare'
    return location

In [ ]:
remap_location(hp_2d,location='milton')

'Rare'

In [ ]:
remap_location(hp_2d,location='Seattle')

'Seattle'

In [ ]:
hp_2d['city'] = hp_2d['city'].apply(lambda x: remap_location(hp_2d, location=x))
hp_2d['city']

0       Shoreline
1         Seattle
2            Kent
3        Bellevue
4         Redmond
          ...    
4595      Seattle
4596     Bellevue
4597       Renton
4598      Seattle
4599         Rare
Name: city, Length: 4600, dtype: object

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

transformer = make_column_transformer(
    (MinMaxScaler(), ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement', 'house_age']),
    (OneHotEncoder(handle_unknown='ignore'), ['bedrooms', 'bathrooms', 'floors', 'view', 'condition'])
)

In [ ]:
from sklearn.model_selection import train_test_split
X=hp_2d.drop('price',axis=1)
Y=hp_2d['price']

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,train_size=0.8,random_state=42)

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((3680, 15), (920, 15), (3680,), (920,))

In [ ]:
# Fit on the train set
transformer.fit(X_train)

# Apply the transformation
X_train_tf = transformer.transform(X_train)
X_test_tf = transformer.transform(X_test)

In [ ]:
X_train_tf.shape, X_test_tf.shape

((3680, 55), (920, 55))

In [ ]:
X_train_tf

<3680x55 sparse matrix of type '<class 'numpy.float64'>'
	with 34559 stored elements in Compressed Sparse Row format>

In [ ]:
X_train_tf.toarray()

array([[0.24819028, 0.04180033, 0.31372549, ..., 0.        , 1.        ,
        0.        ],
       [0.34643226, 0.02645821, 0.4379085 , ..., 1.        , 0.        ,
        0.        ],
       [0.25232678, 0.00976359, 0.31895425, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.08066184, 0.00699249, 0.08104575, ..., 0.        , 1.        ,
        0.        ],
       [0.05584281, 0.00127052, 0.07058824, ..., 1.        , 0.        ,
        0.        ],
       [0.11582213, 0.00296857, 0.06405229, ..., 1.        , 0.        ,
        0.        ]])

In [ ]:
X_train_tf_2A=X_train_tf.toarray()
X_test_tf_2A=X_test_tf.toarray()

In [ ]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import backend as K

In [ ]:
def rmse(Y_true,Y_pred):
  return K.sqrt(K.mean(K.square(Y_pred-Y_true)))

In [ ]:
tf.random.set_seed(42)

model = Sequential([
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(1)
])

model.compile(
    loss=rmse,
    optimizer=Adam(),
    metrics=[rmse]
)

model.fit(X_train_tf_2A, Y_train, epochs=100)

Epoch 1/100
115/115 [==============================] - 3s 2ms/step - loss: 645256.4375 - rmse: 645256.4375
Epoch 2/100
115/115 [==============================] - 0s 2ms/step - loss: 472775.9375 - rmse: 472775.9375
Epoch 3/100
115/115 [==============================] - 0s 2ms/step - loss: 320366.5000 - rmse: 320366.5000
Epoch 4/100
115/115 [==============================] - 0s 2ms/step - loss: 302853.2812 - rmse: 302853.2812
Epoch 5/100
115/115 [==============================] - 0s 2ms/step - loss: 289253.3125 - rmse: 289253.3125
Epoch 6/100
115/115 [==============================] - 0s 2ms/step - loss: 283768.6875 - rmse: 283768.6875
Epoch 7/100
115/115 [==============================] - 0s 2ms/step - loss: 276237.8750 - rmse: 276237.8750
Epoch 8/100
115/115 [==============================] - 0s 2ms/step - loss: 269233.0000 - rmse: 269233.0000
Epoch 9/100
115/115 [==============================] - 0s 2ms/step - loss: 267987.9062 - rmse: 267987.9062
Epoch 10/100
115/115 [===============

In [ ]:
predictions=model.predict(X_test_tf_2A)

In [ ]:
predictions[:5]

array([[ 319182.3 ],
       [ 352194.88],
       [1264410.1 ],
       [ 510887.6 ],
       [ 374565.66]], dtype=float32)

In [ ]:
predictions_1A=np.ravel(predictions)
predictions_1A[0:5]

array([ 319182.3 ,  352194.88, 1264410.1 ,  510887.6 ,  374565.66],
      dtype=float32)

In [ ]:
rmse(Y_test,predictions_1A).numpy()

992082.102661593